# TO-DO's


In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta
import numpy as np
import os
import math
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Don't wrap columns
pd.set_option('display.max_colwidth', None) # Show full content of each column
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
Coin = ["BTC-USD", "ETH-USD", "USDT-USD", "BNB-USD", "USDC-USD", "XRP-USD", "DOGE-USD", "ADA-USD","SOL-USD", "DOT-USD", "MATIC-USD", "LTC-USD", "SHIB-USD", "BCH-USD", "LINK-USD", "XLM-USD","AVAX-USD", "ATOM-USD", "HBAR-USD", "FIL-USD", "TRX-USD", "APE-USD", "ICP-USD", "QNT-USD","NEAR-USD", "LDO-USD", "ALGO-USD", "ARB-USD", "EGLD-USD", "VET-USD", "MANA-USD", "FTM-USD","XTZ-USD", "AAVE-USD", "SAND-USD", "STX-USD", "EOS-USD", "CAKE-USD", "KAVA-USD", "CHZ-USD", "THETA-USD", "ENJ-USD", "WAVES-USD", "RUNE-USD", "1INCH-USD", "BAT-USD","OMG-USD", "ZIL-USD", "CRO-USD", "GALA-USD", "YFI-USD", "CRV-USD", "ANKR-USD","SNX-USD", "UNI-USD", "REN-USD", "CELR-USD", "ONT-USD", "RVN-USD", "DASH-USD", "KLAY-USD","FTT-USD", "MINA-USD", "GLMR-USD", "DYDX-USD", "MKR-USD", "ZRX-USD", "BAND-USD","BAL-USD", "ROSE-USD", "FLR-USD", "MOVR-USD", "OP-USD", "TWT-USD", "WOO-USD", "NEXO-USD","HOT-USD", "SUSHI-USD", "SRM-USD", "CVC-USD", "LRC-USD", "QTUM-USD", "ICX-USD", "XEC-USD","RLC-USD", "LSK-USD", "GNO-USD", "STEEM-USD", "WAXP-USD", "SXP-USD", "VTHO-USD", "RENBTC-USD","SPELL-USD", "ILV-USD", "AKT-USD", "PENDLE-USD", "NEO-USD", "AERGO-USD"]
portfolio = ["BTC-USD", "ETH-USD", "USDT-USD", "BNB-USD", "USDC-USD"]

In [ ]:
################################################CLASSIC STOCK DATA ANALYSIS###################################################
#bist = endeks
def classic_analysis(bist):
    stock_dict = {}
    stock_data = []
    stock_number = 0
    for stocks in bist:
        try:
            ticker = yf.Ticker(stocks)
            stock_info = ticker.info
            data = {key: stock_info.get(key, None) for key in ["symbol","regularMarketPrice", "priceToBook", "currentPrice", "targetHighPrice", "targetLowPrice", "targetMeanPrice", "targetMedianPrice",
                                                                "bookValue", "open", "dayLow", "dayHigh", "recommendationKey", 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh']}
            historical_data = ticker.history(period="1y",interval="1h")  # Get historical data for the stock
            stock_dict[stocks] = historical_data
            if historical_data.empty:
                print(f"No historical data for {stocks}, skipping...")
                continue  # Skip this ticker if no historical data is found
            stock_number = 1 + stock_number
            print(f"\r{stock_number}/{len(bist)} Downloaded {stocks} data", end='', flush=True)

            historical_data['RSI'] = ta.rsi(historical_data['Close'], length=14)  # Calculate RSI (Relative Strength Index)
            adx_data = ta.adx(historical_data['High'], historical_data['Low'], historical_data['Close'], length=14)  # Calculate ADX (Average Directional Index)
            historical_data['ADX'] = adx_data['ADX_14']
            macd = ta.macd(historical_data['Close'], fast=12, slow=26, signal=9)
            historical_data = pd.concat([historical_data, macd], axis=1)
            # Indicators
            
            historical_data['MACDAS'] = historical_data['MACD_12_26_9'] - historical_data['MACDs_12_26_9']
            historical_data['MACDAS_Signal'] = historical_data['MACDAS'].ewm(span=9, adjust=False).mean()
            historical_data['CCI'] = ta.cci(historical_data['High'], historical_data['Low'], historical_data['Close'], length=20)  # Calculate CCI (Commodity Channel Index)
            historical_data['ROC'] = ta.roc(historical_data['Close'], length=12)  # Calculate ROC (Rate of Change)
            historical_data['ATR'] = ta.atr(historical_data['High'], historical_data['Low'], historical_data['Close'], length=14)  # Calculate ATR (Average True Range)
            historical_data['MFI'] = ta.mfi(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'], length=14)
            historical_data["FWMA"]= ta.fwma(historical_data['Close'],length=14)
            historical_data['OBV'] = ta.obv(historical_data['Close'], historical_data['Volume'])  # Calculate OBV (On-Balance Volume)
            historical_data['CMF'] = ta.cmf(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])# Calculate CMF (Chaikin Money Flow)
            historical_data['AD'] =  ta.ad(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])  # Calculate A/D Line (Accumulation/Distribution Line)
            historical_data["VWAP"]= ta.vwap(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])
            historical_data["kama"]= ta.kama(historical_data['Close'],length=14) # lenght
            supertrend = ta.supertrend(historical_data['High'], historical_data['Low'], historical_data['Close'], length=10, multiplier=3)
            historical_data['Supertrend'] = supertrend['SUPERT_10_3.0']
            historical_data['Supertrend_direction'] = supertrend['SUPERTd_10_3.0']
            
            
            # Calculate Stochastic Oscillator
            stoch = ta.stoch(historical_data['High'], historical_data['Low'], historical_data['Close'], k=14, d=3)
            historical_data['Stoch_K'] = stoch['STOCHk_14_3_3']
            historical_data['Stoch_D'] = stoch['STOCHd_14_3_3']
            
            # Calculate Stochastic RSI
            stochrsi = ta.stochrsi(historical_data['Close'], length=14, rsi_length=14, k=3, d=3)
            historical_data['StochRSI_K'] = stochrsi['STOCHRSIk_14_14_3_3']
            historical_data['StochRSI_D'] = stochrsi['STOCHRSId_14_14_3_3']
 
            # Calculate Bollinger Bands
            bollinger = ta.bbands(historical_data['Close'], length=20, std=2)
            historical_data['BB_Middle'] = bollinger.iloc[:, 1]  # Middle Band
            historical_data['BB_Upper'] = bollinger.iloc[:, 2]  # Upper Band
            historical_data['BB_Lower'] = bollinger.iloc[:, 0]  # Lower Band
            historical_data['BB_BWidth'] = bollinger.iloc[:, 3]  # Bandwidth
            historical_data['BB_%B'] = bollinger.iloc[:, 4]  # %B

            data.update({"RSI": historical_data['RSI'].iloc[-1], "ADX": historical_data['ADX'].iloc[-1],"CCI": historical_data['CCI'].iloc[-1], "ROC": historical_data['ROC'].iloc[-1], "ATR": historical_data['ATR'].iloc[-1], "OBV": historical_data['OBV'].iloc[-1],
                        "CMF": historical_data['CMF'].iloc[-1], "AD": historical_data['AD'].iloc[-1], "MFI": historical_data['MFI'].iloc[-1],
                        "MACD": historical_data['MACD_12_26_9'].iloc[-1],"MACD_signal": historical_data['MACDs_12_26_9'].iloc[-1],"MACD_Hist": historical_data['MACDh_12_26_9'].iloc[-1],"MACDAS": historical_data['MACDAS'].iloc[-1],"MACDAS_Signal": historical_data['MACDAS_Signal'].iloc[-1],
                        "BB_Middle": historical_data['BB_Middle'].iloc[-1],"BB_Upper": historical_data['BB_Upper'].iloc[-1],"BB_Lower": historical_data['BB_Lower'].iloc[-1],'BB_%B': historical_data['BB_%B'].iloc[-1],'BB_BWidth': historical_data['BB_BWidth'].iloc[-1],
                        "FWMA": historical_data["FWMA"].iloc[-1],"VWAP": historical_data["VWAP"].iloc[-1],
                        "kama" : historical_data["kama"].iloc[-1],
                        "Supertrend": historical_data['Supertrend'].iloc[-1],
                        "Supertrend_direction": historical_data['Supertrend_direction'].iloc[-1],
                        "Stoch_K": historical_data['Stoch_K'].iloc[-1],
                        "Stoch_D": historical_data['Stoch_D'].iloc[-1],
                        "StochRSI_K": historical_data['StochRSI_K'].iloc[-1],
                        "StochRSI_D": historical_data['StochRSI_D'].iloc[-1],
                        })
            stock_data.append(data)

        except Exception as e:
            print(f"Error fetching data for {stocks}: {e}")

    # Create a DataFrame from the stock data
    df = pd.DataFrame(stock_data)
    df["currentPrice"] = df["regularMarketPrice"]
    df["YF%"] = ((df["currentPrice"] - df["fiftyTwoWeekLow"])/(df["fiftyTwoWeekHigh"]-df["fiftyTwoWeekLow"]))*100 #Check the differences
    df["MACDAS-dif"] = df["MACDAS"] - df["MACDAS_Signal"] 
    df["change"] = ((df["regularMarketPrice"] / df["open"]) - 1) * 100
    #df["change"] = ((df["currentPrice"] / df["open"]) - 1) * 100
    df["BB_Pot"] = ((df['BB_Upper'] / df["currentPrice"]) - 1) * 100
    df["BB_Opt"] = ((df['BB_Lower'] / df["currentPrice"]) - 1) * 100
    df["TrendWay"] = np.select([(df["ADX"] > 20) & (df["ROC"] > 0), (df["ADX"] > 20) & (df["ROC"] <= 0), (df["ADX"] <= 20)], ["upper", "lower", "no-trend"], default="unknown")

    df1 = df[["symbol", "Stoch_K","Stoch_D","StochRSI_K","StochRSI_D","MACDAS-dif","MACDAS","MACDAS_Signal", 'BB_BWidth', 'BB_%B',"YF%","MFI", "OBV", "RSI","AD", "ADX", "CCI", "ROC","CMF", "ATR","FWMA","VWAP","Supertrend","Supertrend_direction","kama", "currentPrice","change"]]
    df2 = df1.rename(columns={"currentPrice": "CPrice","Supertrend_direction":"st_way"})

    pd.set_option('display.float_format', '{:.2f}'.format)
    print("\nDownload process is done!")
    df2
    now = datetime.now()  # This part will copy for our sell lists. Especially further analysis.
    formatted_time = now.strftime("%m-%d_%H-%M-%S")
    #df2.to_csv(f"CSVs/Results/{formatted_time}.csv")
    df2.to_csv(f"CSVs/Coin_last.csv")
    print(df2)
#classic_analysis(portfolio)

In [ ]:
classic_analysis(portfolio) 

In [ ]:
df2 = pd.read_csv(f"CSVs/Coin_last.csv")
df2

In [ ]:
cheap = df2.copy()                         #we can catch early bullish stocks with this sample
# CMF  C. Money Flow index important
# ATR  Average True Range but I don't know the real value	
# FWMA	Fibonacci
# VWAP	Volume-weighted average price I don't know 
cheap = cheap[cheap["CCI"] < -80]
#cheap = cheap[cheap["MFI"] < 30] # Money Flow Index
cheap = cheap[cheap["RSI"] < 40]
cheap = cheap[cheap["YF%"] < 40]
cheap = cheap[cheap["ADX"] < 25]
cheap = cheap[cheap["BB_%B"] < 0.4] # We can miss the upward movements 
#cheap = cheap[cheap["ROC"] > 0] # Momentum tool (rate of change) 

#cheap = cheap[cheap["MACDAS"] > cheap["MACDAS_Signal"]]
#cheap = cheap[cheap["Stoch_K"] < cheap["Stoch_D"]]
cheap.head(50)
cheap["time"] = datetime.now()
cheap = cheap[["symbol",'BB_BWidth', 'BB_%B',"YF%","MFI","RSI","ADX", "CCI", "ROC","CMF", "ATR","FWMA","VWAP","st_way","Supertrend","kama","CPrice","change","time"]]
cheap = cheap.sort_values(by=["time","RSI"])
#cheap.to_csv(f"CSVs/Cheap.csv")

file_path = "CSVs/CheapCoin.csv"

if os.path.exists(file_path):
    old_data = pd.read_csv(file_path)
    combined = pd.concat([old_data, cheap], ignore_index=True)
else:
    combined = cheap

# Save the updated file
combined.to_csv(file_path, index=False)
cheap

In [ ]:
daily_trades = df2.copy()  # Create a copy of the dataframe for daily trading signals

# Filter for daily trading signals - focusing on momentum and trend indicators
# ADX > 20 indicates a stronger trend
# ROC > 0 indicates positive momentum
# Check if price is above VWAP (bullish signal for day trading)
# Check if price is crossing above key moving average (kama)

daily_trades = daily_trades[daily_trades["ADX"] > 20]  # Strong trend
daily_trades = daily_trades[daily_trades["ROC"] > 0]   # Positive momentum
#daily_trades = daily_trades[daily_trades["CPrice"] > daily_trades["VWAP"]]  # Price above VWAP
daily_trades = daily_trades[daily_trades["st_way"] > 0]  # Supertrend indicator is positive

# Add a volatility filter - we want enough movement for day trading
daily_trades = daily_trades[daily_trades["BB_BWidth"] > 1.5]  # Bollinger Band width indicating volatility

# Look for recent bullish reversal signals
daily_trades = daily_trades[daily_trades["RSI"] > 40]  # Not oversold, starting to move up
daily_trades = daily_trades[daily_trades["RSI"] < 70]  # Not overbought yet

# Sort results by momentum and trend strength
daily_trades["score"] = (daily_trades["ROC"] * 0.4) + \
                         (daily_trades["ADX"] * 0.3) + \
                         ((daily_trades["RSI"] - 40) * 0.3)  # Custom scoring system

# Mark timestamp
daily_trades["time"] = datetime.now()

# Select relevant columns and sort
daily_trades = daily_trades[["symbol", 'BB_BWidth', 'BB_%B', "YF%", "MFI", "RSI", "ADX", 
                             "CCI", "ROC", "CMF", "ATR", "FWMA", "VWAP", "st_way", 
                             "Supertrend", "kama", "CPrice", "change", "score", "time"]]
daily_trades = daily_trades.sort_values(by=["score"], ascending=False)

## Create a folder if it doesn't exist
#os.makedirs("CSVs", exist_ok=True)
#
# Save data to CSV with timestamp to track daily signals
#timestamp = datetime.now().strftime("%Y%m%d")
#file_path = f"CSVs/DailyTrades_{timestamp}.csv"
#
## Update or create the CSV file
#if os.path.exists(file_path):
#    old_data = pd.read_csv(file_path)
#    combined = pd.concat([old_data, daily_trades], ignore_index=True)
#    # Remove duplicates based on symbol (keep the most recent)
#    combined = combined.sort_values('time').drop_duplicates(subset=['symbol'], keep='last')
#else:
#    combined = daily_trades
#
## Save the updated file
#combined.to_csv(file_path, index=False)
#
## Also maintain a historical log of all signals
#history_file = "CSVs/DailyTradesHistory.csv"
#if os.path.exists(history_file):
#    history_data = pd.read_csv(history_file)
#    history_combined = pd.concat([history_data, daily_trades], ignore_index=True)
#else:
#    history_combined = daily_trades
#
#history_combined.to_csv(history_file, index=False)
#
## Print top 10 trading candidates
#print(f"Top daily trading candidates for {timestamp}:")
#display(daily_trades.head(10))
daily_trades.head(20)

In [ ]:
# we can use this function for similar metrics like rsi and mfi 
condition = (df2['CCI'] < -100) | (df2['RSI'] < 30) | (df2['YF%'] < -20) | (df2['MFI'] < 20)
filtered_df = df2[condition].copy()
filtered_df

In [ ]:
# Momentum stocks filter
momentum = df2.copy()
momentum = momentum[(momentum["RSI"] >=50) &(momentum["RSI"]< 70)]  # Strong momentum
#momentum = momentum[momentum["MACDAS"] > momentum["MACDAS_Signal"]]  # MACD above signal line
momentum = momentum[momentum["ROC"] > 0]  # Positive rate of change
momentum = momentum[momentum["ADX"] > 20]  # Strong trend
momentum = momentum[momentum["CCI"] < 100]  # Strong trend
#momentum = momentum[momentum["BB_%B"] > 0.5]
momentum = momentum[momentum["kama"] < momentum["CPrice"]]
momentum = momentum[momentum["Supertrend"] < momentum["CPrice"]]  # MACD above signal line
momentum = momentum[momentum["Stoch_K"] < momentum["Stoch_D"]]
momentum = momentum.sort_values(by="ROC", ascending=False)  # Sort by strongest momentum
momentum = momentum[["symbol",'BB_BWidth', 'BB_%B',"YF%","MFI","RSI","ADX", "CCI", "ROC","CMF", "ATR","FWMA","VWAP","st_way","Supertrend","kama","CPrice","change"]]
momentum.head(50)

In [ ]:
top_performers = df2.copy()
top_performers = top_performers[top_performers["change"]>= 6]
top_performers

In [ ]:
#If this one works write portfolio as same format.
XU100 = df2.copy()
XU100 = XU100[XU100["symbol"].isin(bist100)]
XU100 = XU100.sort_values(by="RSI", ascending=True)
XU100.head(50)
XU100.to_csv(f"CSVs/XU100 last.csv")

In [ ]:
#gemini üzerinden seçtiğimiz hisselerin haberlerini getirecek bir kod yazabiliriz.
# Change'e göre sıralayıp listeyi inceleyelim
hisse = "EGPRO.IS"
solo_stock = df2.copy()
solo_stock = solo_stock[solo_stock["symbol"]==hisse]
#print(solo_stock)
solo_stock

In [ ]:
def find_value_opportunities(df, save_history=True):
    """
    Identify potentially undervalued stocks using technical indicators and multi-factor approach.
    
    Parameters:
    df - DataFrame with technical indicators
    save_history - Whether to save results to historical file for tracking
    
    Returns:
    DataFrame of filtered value opportunities
    """
    # Create deep copy to avoid modifying original data
    value_opps = df.copy()
    
    # Create composite technical score using multiple factors
    # This helps prevent false signals from any single indicator
    value_opps['oversold_score'] = 0
    
    # Add points to score based on oversold conditions
    value_opps.loc[value_opps["CCI"] < -100, 'oversold_score'] += 2  # Deeply oversold
    value_opps.loc[(value_opps["CCI"] >= -100) & (value_opps["CCI"] < -80), 'oversold_score'] += 1  # Moderately oversold
    
    value_opps.loc[value_opps["RSI"] < 30, 'oversold_score'] += 2  # Deeply oversold
    value_opps.loc[(value_opps["RSI"] >= 30) & (value_opps["RSI"] < 40), 'oversold_score'] += 1  # Moderately oversold
    
    value_opps.loc[value_opps["MFI"] < 20, 'oversold_score'] += 2  # Deeply oversold
    value_opps.loc[(value_opps["MFI"] >= 20) & (value_opps["MFI"] < 30), 'oversold_score'] += 1  # Moderately oversold
    
    # Bollinger Band related signals
    value_opps.loc[value_opps["BB_%B"] < 0.2, 'oversold_score'] += 1  # Price near lower band
    
    # Filter for stocks with meaningful oversold scores
    value_opps = value_opps[value_opps['oversold_score'] >= 3]
    
    # Check for reversal signals that indicate potential bottoming
    value_opps['reversal_signals'] = 0
    
    # MACD crossover signal (early momentum change)
    value_opps.loc[value_opps["MACDAS"] > value_opps["MACDAS_Signal"], 'reversal_signals'] += 1
    
    # Stochastic reversal (momentum change)
    if "Stoch_K" in value_opps.columns and "Stoch_D" in value_opps.columns:
        value_opps.loc[value_opps["Stoch_K"] > value_opps["Stoch_D"], 'reversal_signals'] += 1
    
    # Volume confirmation (accumulation indicates smart money buying)
    value_opps.loc[value_opps["CMF"] > 0, 'reversal_signals'] += 1  # Positive money flow
    
    # Give preference to stocks showing signs of reversal
    value_opps = value_opps.sort_values(by=["oversold_score", "reversal_signals", "RSI"], ascending=[False, False, True])
    
    # Add timing information
    value_opps["scan_time"] = datetime.now()
    
    # Select relevant columns for output
    result = value_opps[["symbol", "oversold_score", "reversal_signals", 'BB_BWidth', 'BB_%B', 
                        "YF%", "MFI", "RSI", "ADX", "CCI", "ROC", "CMF", "ATR",
                        "FWMA", "VWAP", "st_way", "Supertrend", "kama", "CPrice", 
                        "change", "scan_time"]]
    
    # Save historical data for tracking performance over time
    if save_history:
        file_path = "CSVs/ValueOpportunities.csv"
        
        if os.path.exists(file_path):
            try:
                old_data = pd.read_csv(file_path)
                
                # Keep only the most recent 90 days of data to prevent file bloat
                if 'scan_time' in old_data.columns:
                    old_data['scan_time'] = pd.to_datetime(old_data['scan_time'])
                    cutoff_date = datetime.now() - timedelta(days=90)
                    old_data = old_data[old_data['scan_time'] > cutoff_date]
                
                combined = pd.concat([old_data, result], ignore_index=True)
            except Exception as e:
                print(f"Error reading history file: {e}")
                combined = result
        else:
            combined = result
            
        # Save the updated file
        combined.to_csv(file_path, index=False)
    
    return result
find_value_opportunities(df2)

In [ ]:
def backtest_value_strategy(stock_symbols, lookback_period='1y', holding_period=30):
    """
    Backtest the value opportunity strategy
    
    Parameters:
    stock_symbols - List of stock symbols to test
    lookback_period - Period of historical data to use
    holding_period - How many days to hold stocks after signal
    
    Returns:
    DataFrame with backtest results
    """
    results = []
    
    for symbol in stock_symbols:
        try:
            # Get historical data
            ticker = yf.Ticker(symbol)
            hist_data = ticker.history(period=lookback_period)
            
            if hist_data.empty:
                continue
                
            # Calculate all technical indicators as in your classic_analysis function
            hist_data['RSI'] = ta.rsi(hist_data['Close'], length=14)
            # Add all other indicators from your original code...
            
            # Get signals using your value criteria
            signals = pd.DataFrame(index=hist_data.index)
            signals['symbol'] = symbol
            signals['price'] = hist_data['Close']
            
            # Define entry condition (equivalent to the value_opportunities function)
            signals['oversold'] = ((hist_data['CCI'] < -80) & 
                                  (hist_data['RSI'] < 35))
            
            # Find signal dates
            signal_dates = signals[signals['oversold']].index
            
            # Calculate returns for holding period after each signal
            for date in signal_dates:
                try:
                    entry_price = hist_data.loc[date, 'Close']
                    exit_idx = min(hist_data.index.get_loc(date) + holding_period, len(hist_data) - 1)
                    exit_date = hist_data.index[exit_idx]
                    exit_price = hist_data.loc[exit_date, 'Close']
                    
                    pct_return = (exit_price / entry_price - 1) * 100
                    
                    results.append({
                        'symbol': symbol,
                        'entry_date': date,
                        'exit_date': exit_date,
                        'entry_price': entry_price,
                        'exit_price': exit_price,
                        'return': pct_return
                    })
                except Exception as e:
                    print(f"Error calculating returns for {symbol} on {date}: {e}")
                    
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
            
    return pd.DataFrame(results)
backtest_value_strategy(portfolio)